In [1]:
import nltk
from nltk import RegexpParser

In [1]:
string = 'Already, we have done more to advance the cause of health care reform in the last thirty days than we have in the last decade.  When it was days old, this Congress passed a law to provide and protect health insurance for eleven million American children whose parents work full-time.  Our recovery plan will invest in electronic health records and new technology that will reduce errors, bring down costs, ensure privacy, and save lives.  It will launch a new effort to conquer a disease that has touched the life of nearly every American by seeking a cure for cancer in our time.  And it makes the largest investment ever in preventive care, because that is one of the best ways to keep our people healthy and our costs under control.'
string2 = 'A future of hope and opportunity requires that all our citizens have affordable and available health care. When it comes to health care, government has an obligation to care for the elderly, the disabled, and poor children, and we will meet those responsibilities. For all other Americans, private health care insurance is the best way to meet their needs. But many Americans cannot afford a health insurance policy.'

In [3]:
sentences = nltk.sent_tokenize(string)
sentences = [nltk.word_tokenize(sent) for sent in sentences]
sentences = [nltk.pos_tag(sent) for sent in sentences]

In [4]:
sentences

[[('Our', 'PRP$'),
  ('deep', 'JJ'),
  ('and', 'CC'),
  ('heartfelt', 'JJ'),
  ('gratitude', 'NN'),
  ('is', 'VBZ'),
  ('due', 'JJ'),
  ('to', 'TO'),
  ('that', 'DT'),
  ('Almighty', 'NNP'),
  ('Power', 'NNP'),
  ('which', 'WDT'),
  ('has', 'VBZ'),
  ('bestowed', 'VBN'),
  ('upon', 'IN'),
  ('us', 'PRP'),
  ('such', 'JJ'),
  ('varied', 'JJ'),
  ('and', 'CC'),
  ('numerous', 'JJ'),
  ('blessings', 'NNS'),
  ('throughout', 'IN'),
  ('the', 'DT'),
  ('past', 'JJ'),
  ('year', 'NN'),
  ('.', '.')],
 [('The', 'DT'),
  ('general', 'JJ'),
  ('health', 'NN'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('country', 'NN'),
  ('has', 'VBZ'),
  ('been', 'VBN'),
  ('excellent', 'JJ'),
  (',', ','),
  ('our', 'PRP$'),
  ('harvests', 'NNS'),
  ('have', 'VBP'),
  ('been', 'VBN'),
  ('unusually', 'RB'),
  ('plentiful', 'JJ'),
  (',', ','),
  ('and', 'CC'),
  ('prosperity', 'NN'),
  ('smiles', 'NNS'),
  ('throughout', 'IN'),
  ('the', 'DT'),
  ('land', 'NN'),
  ('.', '.')],
 [('Indeed', 'RB'),
  (',', ','),
  ('

In [5]:
grammar = "NP: {<DT>?<JJ.*>*<IN>?<NN.*>+}"

In [6]:
cp = nltk.RegexpParser(grammar)
for i in sentences:
    result = cp.parse(i)
    print(result)

(S
  Our/PRP$
  deep/JJ
  and/CC
  (NP heartfelt/JJ gratitude/NN)
  is/VBZ
  due/JJ
  to/TO
  (NP that/DT Almighty/NNP Power/NNP)
  which/WDT
  has/VBZ
  bestowed/VBN
  upon/IN
  us/PRP
  such/JJ
  varied/JJ
  and/CC
  (NP numerous/JJ blessings/NNS)
  throughout/IN
  (NP the/DT past/JJ year/NN)
  ./.)
(S
  (NP The/DT general/JJ health/NN)
  of/IN
  (NP the/DT country/NN)
  has/VBZ
  been/VBN
  excellent/JJ
  ,/,
  our/PRP$
  (NP harvests/NNS)
  have/VBP
  been/VBN
  unusually/RB
  plentiful/JJ
  ,/,
  and/CC
  (NP prosperity/NN smiles/NNS)
  throughout/IN
  (NP the/DT land/NN)
  ./.)
(S
  Indeed/RB
  ,/,
  notwithstanding/VBG
  our/PRP$
  (NP demerits/NNS)
  ,/,
  we/PRP
  have/VBP
  (NP much/JJ reason/NN)
  to/TO
  believe/VB
  from/IN
  (NP the/DT past/JJ events/NNS)
  in/IN
  our/PRP$
  (NP history/NN)
  that/IN
  we/PRP
  have/VBP
  enjoyed/VBN
  (NP the/DT special/JJ protection/NN)
  (NP of/IN Divine/NNP Providence/NNP)
  ever/RB
  since/IN
  our/PRP$
  (NP origin/NN)
  as/IN
  (N

In [6]:
import spacy
import pandas as pd

In [3]:
def spacy_fxn(strings):
    
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(strings)
    l = []
    for token in doc.noun_chunks:
        l.append(token)
    
    new_dict = {}
    for i in l:
        if str(i) in new_dict:
            new_dict[str(i)] += 1
        else:
            new_dict[str(i)] = 1
    return new_dict

In [4]:
dict1 = spacy_fxn(string)

In [5]:
dict2 = spacy_fxn(string2)

In [20]:
y = pd.DataFrame(dict1, index =[0])

In [21]:
df_asint = y.astype(int)
coocc = df_asint.T.dot(df_asint)
coocc

,we,the cause,health care reform,the last thirty days,the last decade,it,this Congress,a law,health insurance,eleven million American children,...,nearly every American,a cure,cancer,our time,the largest investment,preventive care,the best ways,our people,our costs,control
we,4,2,2,2,2,4,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
the cause,2,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
health care reform,2,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
the last thirty days,2,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
the last decade,2,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
it,4,2,2,2,2,4,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
this Congress,2,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
a law,2,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
health insurance,2,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eleven million American children,2,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [24]:
x = pd.DataFrame(dict2, index =[0])
x

,A future,hope,opportunity,all our citizens,affordable and available health care,it,health care,government,an obligation,poor children,we,those responsibilities,all other Americans,private health care insurance,the best way,their needs,many Americans,a health insurance policy
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [25]:
df_asint1 = x.astype(int)
coocc1 = df_asint1.T.dot(df_asint1)
coocc1

,A future,hope,opportunity,all our citizens,affordable and available health care,it,health care,government,an obligation,poor children,we,those responsibilities,all other Americans,private health care insurance,the best way,their needs,many Americans,a health insurance policy
A future,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
hope,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
opportunity,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
all our citizens,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
affordable and available health care,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
it,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
health care,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
government,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
an obligation,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
poor children,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [27]:
coocc1.add(coocc, fill_value = 0)

,A future,It,Our recovery plan,a cure,a disease,a health insurance policy,a law,a new effort,affordable and available health care,all other Americans,...,the cause,the largest investment,the last decade,the last thirty days,the life,their needs,this Congress,those responsibilities,we,whose parents
A future,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN
It,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,2.0,1.0
Our recovery plan,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,2.0,1.0
a cure,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,2.0,1.0
a disease,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,2.0,1.0
a health insurance policy,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN
a law,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,2.0,1.0
a new effort,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,2.0,1.0
affordable and available health care,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN
all other Americans,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN


In [34]:
nlp = spacy.load('en_core_web_md')  # make sure to use larger model!
tokens = nlp('available and affordable health care')
t = nlp('private health care insurance')

tokens.similarity(t)

OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.